In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
# в течение матча
!cp "/content/drive/MyDrive/NIR/checkpoint-603-20230423T185730Z-002.zip" "/content/checkpoint-603-20230423T185730Z-002.zip"
!unzip "/content/checkpoint-603-20230423T185730Z-002.zip"

Archive:  /content/checkpoint-603-20230423T185730Z-002.zip
  inflating: checkpoint-603/scheduler.pt  
  inflating: checkpoint-603/training_args.bin  
  inflating: checkpoint-603/trainer_state.json  
  inflating: checkpoint-603/rng_state.pth  
  inflating: checkpoint-603/scaler.pt  
  inflating: checkpoint-603/special_tokens_map.json  
  inflating: checkpoint-603/generation_config.json  
  inflating: checkpoint-603/config.json  
  inflating: checkpoint-603/tokenizer_config.json  
  inflating: checkpoint-603/optimizer.pt  
  inflating: checkpoint-603/tokenizer.json  
  inflating: checkpoint-603/pytorch_model.bin  


In [2]:
# обобщающие новости
!cp "/content/drive/MyDrive/NIR/drive-download-20230424T071825Z-001.zip" "/content/drive-download-20230424T071825Z-001.zip"
!unzip "/content/drive-download-20230424T071825Z-001.zip"

Archive:  /content/drive-download-20230424T071825Z-001.zip
  inflating: checkpoint-684/generation_config.json  
  inflating: checkpoint-684/special_tokens_map.json  
  inflating: checkpoint-684/config.json  
  inflating: checkpoint-684/training_args.bin  
  inflating: checkpoint-684/scheduler.pt  
  inflating: checkpoint-684/rng_state.pth  
  inflating: checkpoint-684/scaler.pt  
  inflating: checkpoint-684/trainer_state.json  
  inflating: checkpoint-684/tokenizer_config.json  
  inflating: checkpoint-684/optimizer.pt  
  inflating: checkpoint-684/tokenizer.json  
  inflating: checkpoint-684/pytorch_model.bin  


In [3]:
!pip install --quiet transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 21.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)
from datasets import Dataset, load_dataset
from pprint import pprint

#### Обобщающая новость

In [5]:
tokenizer = AutoTokenizer.from_pretrained("/content/checkpoint-684")

model = AutoModelForSeq2SeqLM.from_pretrained("/content/checkpoint-684")

In [6]:
training_args = Seq2SeqTrainingArguments(
    output_dir=".",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    optim="adafactor",
    logging_steps=100,
    evaluation_strategy='epoch',
    save_strategy="epoch",
    load_best_model_at_end=True,
    predict_with_generate=True
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer
    )

In [15]:
tokenizer.src_lang = 'ru_RU'
tokenizer.tgt_lang = 'ru_RU'

text_column = 'content'
summary_column = 'part_body'

max_source_length = 512
max_target_length = 300
padding = 'max_length'
ignore_pad_token_for_loss = True

def preprocess_function(examples):
  inputs = []

  for i in range(len(examples[text_column])):
      if examples['name'][i] and examples['team1'][i] and examples['team2'][i] and examples[text_column][i]:
          inputs.append('</s>'.join([examples['name'][i], examples['team1'][i], examples['team2'][i], examples[text_column][i]]))

  model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

  return model_inputs

In [16]:
# пример обобщающей новости
text = "43 минута. Еще атака ""Томи"", правым флангом и ... первое предупреждение в ""Волге"" : Алдонин за споры с арбитром его получает. 45 минута. Ого! Вторая желтая карточка Алдонину за две минуты и удаление в ""Волге"" ! Вот вам и второй матч Владимира Сельдякова в главном чемпионате страны... 46 минута. Первая замена в ""Волге"" : Александр Шуленин вместо Шоты Бибилова. 56 минута. гол! 1:0 - голышев . Это Сорокин слева от лицевой прострелил мяч в штрафную, где голышев обыграл Шуленина и протолкнул мяч в сетку ворот Парейко! 58 минута. Первое предупреждение в ""Томи"" : Владимир Кисенков наказан за срыв атаки. 59 минута. И первая замена в ""Томи"" : Павел Игнатович вместо Валерия Сорокина. Странно, а казалось, что Сорокин был лучшим в составе хозяев в эти 60 минут... 63 минута. Вы только посмотрите, кто выходит на поле! Вторая замена в ""Волге"" : Дмитрий Булыкин вместо Дмитрия Сычева! 79 минута. Второе предупреждение в ""Томи"" : Аравин наказан за накладку Каряке. 80 минута. Но сначала вторая замена в ""Томи"" : вместо Павла голышева вышел Михаил Комков. Последовательно снимает с игры лучших игроков в составе своей команды Василий Баскаков... 83 минута. Третья замена в ""Волге"" : Луис Шелтон вместо Петра Польчака. 84 минута. И третья замена в ""Томи"" : Никита Баженов вместо Игоря Портнягина. 88 минута. Третье предупреждение в ""Томи"" : кажется Рыков его получил за срыв атаки. 96 минута. Поздравим томичей со второй победой в чемпионате и первым домашним успехом. Если судьи продолжат удалять соперников ""Томи"" в каждом туре, то сибиряки и за зону еврокубков еще смогут побороться... 97 минута. На этом давайте наш онлайн-репортаж заканчивать. Все на ""Амкар"" - ""Ростов"" и ""Урал"" - ""Локомотив"". Кстати, благодаря Sports.ru вы можете следить за ходом обоих матчей одновременно! С вами был Василий Рябинко, до новых встреч в эфире."
pprint(text, width=150)

('43 минута. Еще атака Томи, правым флангом и ... первое предупреждение в Волге : Алдонин за споры с арбитром его получает. 45 минута. Ого! Вторая '
 'желтая карточка Алдонину за две минуты и удаление в Волге ! Вот вам и второй матч Владимира Сельдякова в главном чемпионате страны... 46 минута. '
 'Первая замена в Волге : Александр Шуленин вместо Шоты Бибилова. 56 минута. гол! 1:0 - голышев . Это Сорокин слева от лицевой прострелил мяч в '
 'штрафную, где голышев обыграл Шуленина и протолкнул мяч в сетку ворот Парейко! 58 минута. Первое предупреждение в Томи : Владимир Кисенков наказан '
 'за срыв атаки. 59 минута. И первая замена в Томи : Павел Игнатович вместо Валерия Сорокина. Странно, а казалось, что Сорокин был лучшим в составе '
 'хозяев в эти 60 минут... 63 минута. Вы только посмотрите, кто выходит на поле! Вторая замена в Волге : Дмитрий Булыкин вместо Дмитрия Сычева! 79 '
 'минута. Второе предупреждение в Томи : Аравин наказан за накладку Каряке. 80 минута. Но сначала вторая з

In [17]:
ds = Dataset.from_dict({
            'name': ['Россия. Премьер-лига'],
            'team1': ['Томь'],
            'team2': ['Волга НН'], 
            'content': [text]
            })

dataset = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [18]:
predict_results = trainer.predict(
    dataset,
    metric_key_prefix="predict",
    max_length=300,
    num_beams=3
)

predictions = []
if trainer.is_world_process_zero():
  if training_args.predict_with_generate:
    predictions = tokenizer.batch_decode(
        predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )
    predictions = [pred.strip() for pred in predictions]

In [19]:
for predict in predictions:
  pprint(predict, width=150)

('«Томь» – «Волга» – 1:0. Голышев забил победный гол. Сегодня в матче 20-го тура чемпионата России «Томь» на стадионе «Труд» в Томске обыграла '
 '«Волгу» из Нижнего Новгорода (1:0).')


#### Новость в течение матча

In [22]:
tokenizer = AutoTokenizer.from_pretrained("/content/checkpoint-603")

model = AutoModelForSeq2SeqLM.from_pretrained("/content/checkpoint-603")

In [23]:
training_args = Seq2SeqTrainingArguments(
    output_dir=".",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    optim="adafactor",
    logging_steps=100,
    evaluation_strategy='epoch',
    save_strategy="epoch",
    load_best_model_at_end=True,
    predict_with_generate=True
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer
    )

In [24]:
tokenizer.src_lang = 'ru_RU'
tokenizer.tgt_lang = 'ru_RU'

text_column = 'content'
summary_column = 'part_body'

max_source_length = 512
max_target_length = 300
padding = 'max_length'
ignore_pad_token_for_loss = True

def preprocess_function(examples):
  inputs = []
  
  for i in range(len(examples[text_column])):
      if examples['name'][i] and examples['team1'][i] and examples['team2'][i] and examples[text_column][i]:
          inputs.append('</s>'.join([examples['name'][i], examples['team1'][i], examples['team2'][i], examples[text_column][i]]))

  model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

  return model_inputs

In [31]:
# пример новости в течение матча
l_name = 'Россия. Премьер-лига'
team1, team2 = 'Химки', 'Торпедо'
text = '45 минута. Эпизод для логопедической тренировки: Антич снес Чурича и получил горчичник. Оба 11-метровых были не явными, но достаточными для назначения. Гости успели отыграться до перерыва, и пока преимущество «Химок» не отражено на табло.'
pprint(text, width=150)

('45 минута. Эпизод для логопедической тренировки: Антич снес Чурича и получил горчичник. Оба 11-метровых были не явными, но достаточными для '
 'назначения. Гости успели отыграться до перерыва, и пока преимущество «Химок» не отражено на табло.')


In [32]:
# пример новости в течение матча
l_name_2 = 'Россия. Премьер-лига'
team1_2, team2_2 = 'Химки', 'Торпедо'
text_2 = "67 минута. Почему их всех держали на лавке?! Какой шикарный удар со штрафного через стенку в самый угол! гол! ГЛАВЧИЧ! 4:2! 70 минута. Супершанс! Андре бил с метра! 72 минута. Турищев и Енин вышли вместо Андре и Караева."
pprint(text_2, width=150)

('67 минута. Почему их всех держали на лавке?! Какой шикарный удар со штрафного через стенку в самый угол! гол! ГЛАВЧИЧ! 4:2! 70 минута. Супершанс! '
 'Андре бил с метра! 72 минута. Турищев и Енин вышли вместо Андре и Караева.')


In [33]:
ds = Dataset.from_dict({
            'name': [l_name, l_name_2],
            'team1': [team1, team1_2],
            'team2': [team2, team2_2], 
            'content': [text, text_2]
            })

dataset = ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [34]:
predict_results = trainer.predict(
    dataset,
    metric_key_prefix="predict",
    max_length=300,
    num_beams=3
)

predictions = []
if trainer.is_world_process_zero():
  if training_args.predict_with_generate:
    predictions = tokenizer.batch_decode(
        predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )
    predictions = [pred.strip() for pred in predictions]

In [35]:
for predict in predictions:
  pprint(predict, width=150)

('«Химки» – «Торпедо». Антич получил травму. Защитник «Торпедо» Антич получил травму в матче с «Химками». Защитник «Химок» Александр Антич получил '
 'травму в первом тайме матча 9-го тура чемпионата России против «Торпедо» (в данный момент счет 0:0). На 45-й минуте встречи Антич столкнулся с '
 'защитником «Торпедо» Николаем Чуричем и получил удар горчичника. Нападающий «Химок» покинул поле на носилках.')
('«Химки» – «Торпедо». Караев получил травму. Полузащитник «Торпедо» Андрей Караев получил травму в матче чемпионата России против «Химок». '
 'Нападающий «Химок» получил повреждение в конце первого тайма матча против «Торпедо» (в данный момент счет 4:2). На 72-й минуте встречи вместо него '
 'на поле вышел Александр Турищев. «Химки» – «Торпедо». Онлайн')
